In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sksurv.ensemble import RandomSurvivalForest

from sklearn.model_selection import GridSearchCV, KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, MinMaxScaler, StandardScaler, RobustScaler
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.compose import ColumnTransformer
from sksurv.svm import FastSurvivalSVM
from sksurv.metrics import integrated_brier_score
import pickle

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")

In [ ]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
import pandas as pd
import numpy as np

# Model selection

In [ ]:
model_string = 'Random regression with all features'

In [ ]:
dataset = pd.read_csv('./data/complete_dataset_v3.csv', index_col = 0)
dataset.head()

In [ ]:
train_data = pd.read_csv('./data/train_data.csv', index_col=0)
test_data = pd.read_csv('./data/test_data.csv', index_col=0)

### drop days_since_last_payment and days_since_last_contact for linear models

In [ ]:
if 'Random' in model_string:
    cols = ['debtor_payment_past_30_days','debtor_payment_past_180_days', 'debtor_payment_past_365_days', 'debtor_payment_in_dataset', 'debtor_contact_past_30_days', 'debtor_contact_past_180_days', 'debtor_contact_past_365_days', 'debtor_contact_in_dataset']
    train_data = train_data.drop(cols, axis = 1)
    test_data = test_data.drop(cols, axis = 1)
else:
    cols = ['days_since_last_payment', 'days_since_last_contact']
    train_data = train_data.drop(cols, axis = 1)
    test_data = test_data.drop(cols, axis = 1)

In [ ]:
train_data.columns

### Train set

In [ ]:
X_train = train_data.drop(['dossier_nr', 'debiteur_relatie_nr', 'label','duration'], axis = 1)
y_train = train_data.loc[:, ['label', 'duration']]

y_train['label'] = y_train['label'].apply(lambda x: True if x else False)
y_train = np.array(y_train.to_records(index=False))

### Test set

In [ ]:
X_test = test_data.drop(['dossier_nr', 'debiteur_relatie_nr', 'label','duration'], axis = 1)
y_test = test_data.loc[:, ['label', 'duration']]


y_test['label'] = y_test['label'].apply(lambda x: True if x else False)
test_labels = y_test.copy()
y_test = np.array(y_test.to_records(index=False))

# Feature selection

In [ ]:
non_zero_coefs = pd.read_csv('./data/non_zero_coefs.csv', index_col = 0)
non_zero_coefs.head()

In [ ]:
features = non_zero_coefs.abs().sort_values('coefficient', ascending = False)
features;

In [ ]:
features.iloc[:10]

In [ ]:
def contains_number(s):
    return any(i.isdigit() for i in s)

In [ ]:
if contains_number(model_string):
    nr_features = [int(s) for s in model_string.split() if s.isdigit()][0]
    features = features.iloc[:nr_features]
    X_train = X_train.loc[:, features.index.tolist()]
    X_test = X_test.loc[:, features.index.tolist()]

In [ ]:
X_train.columns

In [ ]:
# X_train = X_train.loc[:, ['debt_amount', 'nr_open_cases_in_execution_phase']]
# X_test = X_test.loc[:, ['debt_amount', 'nr_open_cases_in_execution_phase']]

# Model selection

In [ ]:
if 'Cox' in model_string:
    model = CoxnetSurvivalAnalysis()
    estimator_pipe = make_pipeline(
        StandardScaler(),
        model
    )
    param_grid = {
        "coxnetsurvivalanalysis__alphas" : [[np.exp(-5)], [np.exp(-4)],  [np.exp(-3)],  [np.exp(-2)], [np.exp(-1)]],
        "coxnetsurvivalanalysis__l1_ratio" : [0, 0.25, 0.5, 0.75, 1]
    }
    
elif 'Random' in model_string:
    model = RandomSurvivalForest(
       max_features="sqrt",
       n_jobs=-1,
       random_state=42
    )
    estimator_pipe = make_pipeline(
        model
    )
    param_grid = {
        "randomsurvivalforest__n_estimators" : [50, 100, 200],
        "randomsurvivalforest__max_depth" : [3,6],
        "randomsurvivalforest__min_samples_split" : [20, 100, 500],
        "randomsurvivalforest__min_samples_leaf" : [100,200],
        "randomsurvivalforest__max_leaf_nodes" : [50,250],
    }
    
elif 'Fast' in model_string:
    model = FastSurvivalSVM(random_state=42)
    estimator_pipe = make_pipeline(
        StandardScaler(),
        model
    )
    param_grid = {
        "fastsurvivalsvm__max_iter": [10, 25, 50],
#         "fastsurvivalsvm__tol" : [[np.exp(-6)], [np.exp(-4)], [np.exp(-2)]],
        "fastsurvivalsvm__alpha": [np.exp(-5), np.exp(-3), np.exp(-1)],
        "fastsurvivalsvm__rank_ratio": [0, 0.5, 1]
    }

In [ ]:
tscv = TimeSeriesSplit(n_splits = 5)

gcv = GridSearchCV(
    estimator_pipe,
    param_grid=param_grid,
    cv = tscv,
    error_score=0.5,
    n_jobs=-1,
    verbose=10
).fit(X_train, y_train)

In [ ]:
gcv_results = pd.DataFrame(gcv.cv_results_).sort_values(by='rank_test_score')
gcv_results

In [ ]:
if 'Cox'in model_string:
    results = gcv_results.loc[:,['param_coxnetsurvivalanalysis__alphas', 'param_coxnetsurvivalanalysis__l1_ratio', 'mean_test_score']]
    
    results.columns = ['alpha', 'l1_ratio', 'Mean C-statistic of test set' ]
    results.alpha = results.alpha.apply(lambda x: x[0])
print(results.iloc[:5].to_latex(index=False))

In [ ]:
if 'Random' in model_string:
    results = gcv_results.loc[:, ['param_randomsurvivalforest__n_estimators', 'param_randomsurvivalforest__max_depth', 'param_randomsurvivalforest__min_samples_split', 'param_randomsurvivalforest__min_samples_leaf', 'param_randomsurvivalforest__max_leaf_nodes', 'mean_test_score']]
    results.columns = ['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'max_leaf_nodes', 'Mean C-statistic of test set']


In [ ]:
print(results.iloc[:5].to_latex(index=False))

In [ ]:
gcv.best_estimator_

In [ ]:
save_obj(gcv.best_params_, 'rsf_best_params')

In [ ]:
best_params = load_obj('rsf_best_params')

In [ ]:
best_params

In [ ]:
gcv.best_params_

In [ ]:
gcv.best_estimator_.score(X_train, y_train)

In [ ]:
gcv.best_estimator_.score(X_test, y_test)

# Best params training

In [ ]:
if 'Cox' in model_string:
    pipe_pred =  make_pipeline(
        StandardScaler(),
        CoxnetSurvivalAnalysis(fit_baseline_model=True)
    )

In [ ]:
if 'Random' in model_string:
    pipe_pred =  make_pipeline(
        RandomSurvivalForest(
           max_features="sqrt",
           n_jobs=-1,
           random_state=42
        )
    )

In [ ]:
pipe_pred.set_params(**gcv.best_params_)
pipe_pred.fit(X_train, y_train)

# Feautre importance/ coefficients

In [ ]:
if 'Cox' in model_string:
    best_model = pipe_pred.named_steps["coxnetsurvivalanalysis"]
    best_coefs = pd.DataFrame(
        best_model.coef_,
        index=X_train.columns,
        columns=["coefficient"]
    )

    non_zero = np.sum(best_coefs.iloc[:, 0] != 0)
    print("Number of non-zero coefficients: {}".format(non_zero))

    non_zero_coefs = best_coefs.query("coefficient != 0")
    coef_order = non_zero_coefs.abs().sort_values("coefficient").index

    _, ax = plt.subplots(figsize=(12, 6))
    
    non_zero_coefs.loc[coef_order].plot.barh(ax=ax, legend=False)
    ax.set_xlabel("coefficient")
    ax.grid(True)
    if contains_number(model_string):
        plt.title('Coefficients for {}'.format(model_string))
    else:
        plt.title('Coefficients for {}'.format(model_string))
    plt.tight_layout()
    plt.savefig('./figures/appendix/coefficients_{}'.format(model_string))
    
    

In [ ]:
if 'Cox' in model_string:
    best_model = pipe_pred.named_steps["coxnetsurvivalanalysis"]
    best_coefs = pd.DataFrame(
        best_model.coef_,
        index=X_train.columns,
        columns=["coefficient"]
    )

    non_zero = np.sum(best_coefs.iloc[:, 0] != 0)
    print("Number of non-zero coefficients: {}".format(non_zero))

    non_zero_coefs = best_coefs.query("coefficient != 0")
    coef_order = non_zero_coefs.abs().sort_values("coefficient").index

    _, ax = plt.subplots(figsize=(10, 4))
    
    
    non_zero_coefs.loc[coef_order].iloc[len(non_zero_coefs) - 10:].plot.barh(ax=ax, legend=False)
    ax.set_xlabel("coefficient")
    ax.grid(True)
    if contains_number(model_string):
        plt.title('Top 10 coefficients for {}'.format(model_string))
    else:
        plt.title('Top 10 coefficients for {}'.format(model_string))
    plt.tight_layout()
    plt.savefig('./figures/appendix/coefficients_results_{}'.format(model_string))
    
    

In [ ]:
non_zero_coefs.index.name = 'feature'

In [ ]:
pd.option_context("max_colwidth", 1000)

In [ ]:
with pd.option_context("max_colwidth", 1000):
    print(non_zero_coefs.loc[coef_order].iloc[::-1].to_latex())

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

if 'Random' in model_string:    
    perm = PermutationImportance(pipe_pred, n_iter=1, random_state=42)
    perm.fit(X_test, y_test)
    


In [ ]:
if 'Random' in model_string:
    best_coefs = pd.DataFrame(
        perm.feature_importances_,
        index=X_train.columns,
        columns=["coefficient"]
    )

    non_zero = np.sum(best_coefs.iloc[:, 0] != 0)
    print("Number of non-zero coefficients: {}".format(non_zero))

    non_zero_coefs = best_coefs.query("coefficient != 0")
    coef_order = non_zero_coefs.abs().sort_values("coefficient").index

    _, ax = plt.subplots(figsize=(10, 4))
    
    
    non_zero_coefs.loc[coef_order].iloc[len(non_zero_coefs) - 10:].plot.barh(ax=ax, legend=False)
    ax.set_xlabel("relative feature importance")
    ax.grid(True)
    if contains_number(model_string):
        plt.title('Top 10 coefficients for {}'.format(model_string))
    else:
        plt.title('Relative feature importance  for {}'.format(model_string))
    plt.tight_layout()
    plt.savefig('./figures/appendix/coefficients_results_{}'.format(model_string))

In [ ]:
best_coefs.index

In [ ]:
if 'Random' in model_string:
    best_coefs = pd.DataFrame(
        perm.feature_importances_,
        index=X_train.columns,
        columns=["coefficient"]
    )
    non_zero = np.sum(best_coefs.iloc[:, 0] != 0)
    print("Number of non-zero coefficients: {}".format(non_zero))

    non_zero_coefs = best_coefs.query("coefficient > 0")
    coef_order = non_zero_coefs.abs().sort_values("coefficient").index

    _, ax = plt.subplots(figsize=(12, 12))
    
    non_zero_coefs.loc[coef_order].plot.barh(ax=ax, legend=False)
    ax.set_xlabel("relative feature importance")
    ax.grid(True)
    if contains_number(model_string):
        plt.title('Coefficients for {}'.format(model_string))
    else:
        plt.title('Relative feature importance for {}'.format(model_string))
    plt.tight_layout()
    plt.savefig('./figures/appendix/coefficients_{}'.format(model_string))

In [ ]:
with pd.option_context("max_colwidth", 1000):
    print(non_zero_coefs.loc[coef_order].iloc[::-1].to_latex())
    

In [ ]:
# pd.DataFrame(perm.feature_importances_, index = X_train.columns.tolist()).sort_values(by=0)

# Brier score

In [ ]:
lower, upper = np.percentile(y_train["duration"], [0, 90])
times = np.arange(lower, upper + 1)

In [ ]:
model_surv_prob = np.row_stack([
    fn(times)
    for fn in pipe_pred.predict_survival_function(X_test)
])

train_surv_prob = np.row_stack([
    fn(times)
    for fn in pipe_pred.predict_survival_function(X_train)
])

In [ ]:
model_brier = integrated_brier_score(y_train, y_test, model_surv_prob, times)

In [ ]:
test_brier = integrated_brier_score(y_train, y_test, model_surv_prob, times)
train_brier = integrated_brier_score(y_train, y_train, train_surv_prob, times)

In [ ]:
train_brier

In [ ]:
test_brier

In [ ]:
# from sksurv.functions import StepFunction
# from sksurv.nonparametric import kaplan_meier_estimator

# km_func = StepFunction(*kaplan_meier_estimator(y_test["label"], y_test["duration"]))
# km_surv_prob = np.tile(km_func(times), (y_test.shape[0], 1))

In [ ]:
# km_brier = integrated_brier_score(y_train, y_test, km_surv_prob, times)

In [ ]:
# km_brier

In [ ]:
random_surv_prob = 0.5 * np.ones((y_test.shape[0], times.shape[0]))
random_brier = integrated_brier_score(y_train, y_test, random_surv_prob, times)

In [ ]:
random_brier

In [ ]:
train_score = pipe_pred.score(X_train, y_train)
test_score = pipe_pred.score(X_test, y_test)


In [ ]:
pd.DataFrame([[train_score, train_brier, test_score, test_brier]], columns = ['c-index train', 'brier train', 'c-index test', 'brier test'])

In [ ]:
len(X_train.columns)

In [ ]:
X_test

In [ ]:
surv = pipe_pred.predict_survival_function(X_test)

In [ ]:
# surv = pipe_pred.predict_survival_function(X_test.iloc[6000:6006], return_array=True)

# for i, s in enumerate(surv):
#     plt.step(pipe_pred.named_steps['randomsurvivalforest'].event_times_, s, where="post", label=str(i))
# plt.ylabel("Cumulative hazard")
# plt.xlabel("Time in days")
# plt.legend()
# plt.grid(True)

In [ ]:
cum_hazard = pipe_pred.predict_cumulative_hazard_function(X_test, return_array= True)

In [ ]:
# cum_hazard

In [ ]:
test_labels['predicted_tte'] = [np.argmax(x > 1)  if np.amax(x) >= 1 else 10000 for x in cum_hazard ]

In [ ]:
test_labels

In [ ]:
test_labels['predicted_tte'].value_counts()

In [ ]:
predicted_tte = pd.concat([test_data[['days_in_execution_phase', 'dossier_nr']], test_labels], axis=1)
predicted_tte

In [ ]:
event_log = pd.read_csv('./data/dossiers v2 - event log.csv').dropna()
for col in ['datum_van', 'datum_tot', 'datum_actie']:
    event_log[col] = pd.to_datetime(event_log[col])
event_log['days_in_execution_phase'] = (event_log['datum_actie'] - event_log['datum_van']).dt.days
event_log

In [ ]:
(event_log['datum_tot'] - event_log['datum_van']).dt.days

In [ ]:
columns = ['tipping_point', 'total_events', 'total_days', 'total_payments', 'sa_events', 'sa_payments', 'sa_days', 'percentage_events', 'percentage_payments', 'percentage_days']
res = pd.DataFrame(columns = columns)

# range(0, 600, 30)
for tipping_point in range(0, 600, 30):   # [90, 180, 270, 360, 450]
    print("Calculating for {}".format(tipping_point))
    total_days = 0
    total_events = 0
    total_payments = 0
    
    sa_days = 0
    sa_events = 0
    sa_payments = 0

    for i, data in predicted_tte.iterrows():
        filtered_event_log = event_log[(event_log.dossier_nr == data.dossier_nr) & (event_log.days_in_execution_phase >= data.days_in_execution_phase)]
        nr_events = len(filtered_event_log)
        

        total_events += nr_events
        total_payments += data.label
        total_days += data.duration

        if data.predicted_tte < tipping_point:
            sa_events += nr_events
            sa_payments += data.label
            sa_days += data.duration
            
    res = res.append(pd.DataFrame([[tipping_point, total_events, total_payments, total_days, sa_events, sa_payments, sa_days, sa_events/total_events, sa_payments/total_payments, sa_days/total_days]], columns = columns))       
    

In [ ]:
res

In [ ]:
kopy = res.copy()
# kopy = kopy.set_index('tipping_point')
kopy = kopy.melt('tipping_point', var_name='cols', value_name='vals')
kopy =kopy[(kopy['cols'] == 'percentage_events') | (kopy['cols'] == 'percentage_payments')]
kopy['vals'] = kopy['vals'] * 100

In [ ]:
kopy['tipping_point'] = kopy['tipping_point'].astype('int')
kopy['vals'] = pd.to_numeric(kopy['vals'], downcast="float")

In [ ]:
kopy.columns = ['Tipping Point', 'Label', 'Percentage']

In [ ]:
kopy['Label'] = kopy['Label'].apply(lambda x: 'Percentage of actions' if x == 'percentage_events' else 'Percentage of repayments' )

In [ ]:
import seaborn as sns
sns.lineplot(data=kopy[kopy['Tipping Point'] < 600], x='Tipping Point', y='Percentage', hue='Label').set_title('Percentage of events and payments vs chosen mpt')
plt.xlabel('Maximum predicted time')
plt.tight_layout()
plt.savefig('./figures/policy')

In [ ]:
tikkie = res.drop(['total_events', 'total_payments'], axis=1)
tikkie.columns = ['mpt', 'Nr. events', 'Nr. payments', '% events', '% payments']

In [ ]:
tikkie['% events'] = (tikkie['% events'] * 100).round(2)
tikkie['% payments'] = (tikkie['% payments'] * 100).round(2)

In [ ]:
tikkie

In [ ]:
0/0

In [ ]:
tikkie.iloc[2:]['Nr. events'] / tikkie.iloc[2:]['Nr. payments']

In [ ]:
print(tikkie.iloc[1:].to_latex(index=False))